In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pickle
import numpy as np
from pathlib import Path

In [ ]:
# Function to load raw data from pickled objects
def load_data(path: Path):
    pickle_in = open(path, 'rb')
    data_in = pickle.load(pickle_in)
    pickle_in.close()
    return data_in

In [ ]:
root_path = Path('Z:/2_Inscopix/1_DTT/1_OdorAnalysis/') # Root Data Path
contents = list(root_path.iterdir())
folders = [each for each in contents if each.is_dir()] # Get folders for each experiment type
animal_path = [path for folder in folders for path in list(folder.iterdir())] # Get folders for each animal within each experiment
preprocess_path = [path.joinpath('ImagingAnalysis/PreProcessedData') for path in animal_path] # Get PreProcessed Data folder for each animal
auroc_data_path = [path.joinpath('ImagingAnalysis/AUROCData') for path in animal_path]  # Get AUROC Data folder for each animal
odor_data_path = list(preprocess_path[0].glob('*OdorData.pickle'))[0] # Get one OdorFile as all odors are the same

In [ ]:
experiment_names = [folder.name for folder in folders] # Get name of each experiment type
animal_names = np.unique([animal.name for animal in animal_path]) # Get each animal name
plot_names = [f'{animal}-{exp}' for exp in experiment_names for animal in animal_names] # Pair experiements and animals
plot_names = np.reshape(plot_names, (-1, 1)) # put each entry into its own len=1 matrix

In [ ]:
on_time_data_paths = []
latent_data_paths = []

# Loop through the AUROC and PREPROCESS folder to find the files we are interested in. Skip file if there is an error.

for path in auroc_data_path:
    try:
        files = list(path.iterdir())
        
        for file in files:
            file_str = str(file)
            if 'SignificanceTable' in file_str:
                if 'onTime' in file_str:
                    on_time_data_paths.append(file)
                elif 'latent' in file_str:
                    latent_data_paths.append(file)

    except:
        print(f'Error: {str(path)}')
        continue


cell_paths = []
for path in preprocess_path:
    try:
        files = list(path.iterdir())
        for file in files:
            if 'CellList' in str(file):
                cell_paths.append(file)
    except:
        print(f'Error: {str(path)}')
        continue

In [ ]:
# Sort and compile data for plotting

on_time_data = [load_data(path) for path in on_time_data_paths]
latent_data = [load_data(path) for path in latent_data_paths]
cells = [load_data(path) for path in cell_paths]
odor_data = load_data(odor_data_path)
unique_odors = np.unique(odor_data)
num_unique_odors = len(unique_odors)
ratios = tuple([len(cell)/len(cells[0]) for cell in cells])

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

mpl.rcParams['font.family'] = 'Arial'
colormap = ListedColormap(['yellow', 'red', 'green'])

fig, ax = plt.subplot_mosaic(plot_names, gridspec_kw={'height_ratios': ratios, 'hspace': 0}, sharex=True, figsize=(4, 25))
# This took HOURS. We calculate radios based off the number of cells so everything is scaled to the first set of data
# One, common x axis and no space between subplots

for i, label in enumerate(ax): # Loop through each axis (experiment-animal pairing)
    each = ax[label]
    num_cells = len(cells[i])

    each.imshow(latent_data[i], cmap=colormap, extent=(0, num_unique_odors, num_cells, 0))

    each.set_yticks(np.arange(num_cells), labels=[]) # Set major ticks but leave them unlabeled
    each.set_xticks(np.arange(num_unique_odors), labels=[])
    each.set_yticks(np.arange(0.5, num_cells + 0.5, 1), labels=cells[i], minor=True, fontsize=6) # Set minor ticks (offset by 0.5) and label them

    each.set_ylabel(plot_names[i][0], fontsize=7, fontweight='bold') # Set y-axis label to animal-experiment pairing

    each.grid(which='major') # Show the major grid to make nice little squares
    
bottom_ax = ax[plot_names[-1][0]] # Grab the last axis (we will label the x-axis on this one)
bottom_ax.set_xticks(np.arange(0.5, num_unique_odors + 0.5, 1), rotation=90, ha='center', labels=unique_odors, minor=True, fontsize=6, fontweight='bold') # Label the bottommost x-axis with the odors
bottom_ax.set_xlabel('Odors', fontweight = 'bold', fontsize=10)


plt.suptitle('Latent \n Cell v. Odor Significance Matrix', fontsize=16, fontweight='bold') # Set the main title
plt.tight_layout(rect=(0, 0, 1, 0.98)) # Squish the graphs into a box just 'slightly' smaller than max, so theres room for the title
plt.savefig('LatentTimeSignificanceMatrix.pdf', dpi=900) # Save in *Sportscaster Voice* HIGH DEFINITION